In [2]:
# Binance UM Futures: funding rate, open interest, taker long/short ratio (fixed)

import pandas as pd
import requests
from datetime import datetime, timedelta, timezone
from pathlib import Path
import time

FAPI = "https://fapi.binance.com"                # base for fundingRate
DATA = "https://fapi.binance.com/futures/data"   # base for OI & ratios

def to_ms(dt): return int(dt.timestamp() * 1000)

def get_funding(symbol="BTCUSDT", start=None, end=None, sleep=0.1):
    """
    Pull funding rates from /fapi/v1/fundingRate with pagination.
    Max 1000 per call; we loop from start -> end.
    """
    if end is None: end = datetime.now(timezone.utc)
    if start is None: start = end - timedelta(days=90)

    url = f"{FAPI}/fapi/v1/fundingRate"
    params = {"symbol": symbol, "limit": 1000}
    cursor = to_ms(start)
    end_ms = to_ms(end)

    out = []
    while True:
        q = params | {"startTime": cursor, "endTime": end_ms}
        r = requests.get(url, params=q)
        r.raise_for_status()
        batch = r.json()
        if not batch:
            break
        out.extend(batch)
        # advance cursor (last fundingTime + 1 ms)
        cursor = batch[-1]["fundingTime"] + 1
        if cursor > end_ms:
            break
        time.sleep(sleep)  # be gentle

    df = pd.DataFrame(out)
    if df.empty:
        return df
    df["fundingTime"] = pd.to_datetime(df["fundingTime"], unit="ms", utc=True)
    df["fundingRate"] = pd.to_numeric(df["fundingRate"], errors="coerce")
    return df.sort_values("fundingTime").reset_index(drop=True)

def get_oi(symbol="BTCUSDT", period="1h"):
    """
    /futures/data/openInterestHist
    period ∈ {'5m','15m','30m','1h','2h','4h','6h','12h','1d'}
    """
    url = f"{DATA}/openInterestHist"
    r = requests.get(url, params={"symbol": symbol, "period": period, "limit": 500})
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    if df.empty: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    for c in ["sumOpenInterest","sumOpenInterestValue"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df.sort_values("timestamp").reset_index(drop=True)

def get_taker_ratio(symbol="BTCUSDT", period="1h"):
    """
    /futures/data/topLongShortAccountRatio
    period ∈ {'5m','15m','30m','1h','2h','4h','6h','12h','1d'}
    """
    url = f"{DATA}/topLongShortAccountRatio"
    r = requests.get(url, params={"symbol": symbol, "period": period, "limit": 500})
    r.raise_for_status()
    df = pd.DataFrame(r.json())
    if df.empty: return df
    df["timestamp"] = pd.to_datetime(df["timestamp"], unit="ms", utc=True)
    for c in ["longShortRatio","longAccount","shortAccount"]:
        df[c] = pd.to_numeric(df[c], errors="coerce")
    return df.sort_values("timestamp").reset_index(drop=True)

# --- Example pulls ---
funding = get_funding("BTCUSDT")
oi      = get_oi("BTCUSDT", "1h")
taker   = get_taker_ratio("BTCUSDT", "1h")

# --- Save ---
out_dir = Path("binance_derivatives_data/derivatives_raw")

funding.to_csv(out_dir / "binance_funding_BTCUSDT.csv", index=False)
oi.to_csv(out_dir / "binance_open_interest_BTCUSDT.csv", index=False)
taker.to_csv(out_dir / "binance_taker_longshort_BTCUSDT.csv", index=False)

funding.tail(), oi.tail(), taker.tail()

(      symbol                      fundingTime  fundingRate        markPrice
 265  BTCUSDT        2025-08-17 00:00:00+00:00     0.000082  117333.60187083
 266  BTCUSDT 2025-08-17 08:00:00.001000+00:00     0.000047  117949.90000000
 267  BTCUSDT 2025-08-17 16:00:00.007000+00:00     0.000019  118193.93730833
 268  BTCUSDT        2025-08-18 00:00:00+00:00     0.000028  117358.84539167
 269  BTCUSDT 2025-08-18 08:00:00.001000+00:00    -0.000007  115153.20000000,
       symbol  sumOpenInterest  sumOpenInterestValue CMCCirculatingSupply  \
 495  BTCUSDT        88940.708          1.022817e+10    19907843.00000000   
 496  BTCUSDT        88806.452          1.021419e+10    19907843.00000000   
 497  BTCUSDT        88960.407          1.022316e+10    19907843.00000000   
 498  BTCUSDT        89077.621          1.028607e+10    19907843.00000000   
 499  BTCUSDT        89053.108          1.022202e+10    19907843.00000000   
 
                     timestamp  
 495 2025-08-18 10:00:00+00:00  
 496 20